In [1]:
import pandas as pd
import datetime as dt

In [2]:
f = lambda s: dt.datetime.strptime(s,'%d-%m-%Y %H:%M:%S')
df_bn5 = pd.read_csv("data/historical/BankNiftyFut-Sep-5min-2.csv", sep="\t", parse_dates=['Date Time'], date_parser=f, encoding="utf_16_LE") 

#### Remove last empty column and set date time as index

In [3]:
df_bn5 = df_bn5[df_bn5.columns[:-1]]
df_bn5.set_index('Date Time', inplace=True)

#### Apply rolling mean columns for data

In [4]:
df_bn5['SMA6'] = df_bn5['Close'].rolling(6).mean()
df_bn5['SMA21'] = df_bn5['Close'].rolling(21).mean()

## Input the evaluation date below

In [205]:
eval_date = "2020-09-07"

#### Extract a separate dataframe for the evaluation date

In [206]:
today_df_bn5 = df_bn5[eval_date]

In [207]:
today_df_bn5.head(5)

,Open,Low,High,Close,Volume,SMA6,SMA21
Date Time,,,,,,,
2020-09-07 09:15:00,23016.35,22758.00,23062.80,23003.00,370002,23063.208333,23094.071429
2020-09-07 09:20:00,23018.30,22913.00,23026.85,22924.15,197250,23038.658333,23076.219048
2020-09-07 09:25:00,22910.55,22892.80,22950.00,22915.75,155200,23013.158333,23059.364286
2020-09-07 09:30:00,22908.55,22840.05,22908.55,22860.00,156275,22978.158333,23042.273810
2020-09-07 09:35:00,22856.95,22809.75,22890.00,22882.90,155575,22944.966667,23028.040476


### Strategy function

In [208]:
def backtest_Intraday_5min_strategy(df):
    SELL = False
    BUY = False
    SELL_PRICE = 0
    BUY_PRICE = 0
    GAIN_POINTS = 0
    SELL_STOPLOSS = 0
    BUY_STOPLOSS = 0
    STOPLOSS = 100
    SMA6_dead_band = 50
    Total_Signals = 0
    Total_Sell_Signal = 0
    Total_Buy_Signal = 0
    start_time = dt.datetime.strptime("10:00", "%H:%M").time()
    close_time = dt.datetime.strptime("15:15", "%H:%M").time()
    entry_close_time = dt.datetime.strptime("15:00", "%H:%M").time()
    print("Time\t\t\t", "Close\t\t", "SMA6\t\t", "SMA21\t\t", "SIGNAL\t", "TYPE")
    
    for row in df.itertuples():
        
        if row.Index.time() >= start_time and row.Index.time() <= close_time:
            SMA6_upper_band = row.SMA6 + SMA6_dead_band
            SMA6_lower_band = row.SMA6 - SMA6_dead_band
            
            if SELL and row.Close > SELL_STOPLOSS:
                SELL = False
                BUY = False
                Total_Signals += 1
                Total_Buy_Signal += 1
                points = SELL_PRICE - SELL_STOPLOSS
                GAIN_POINTS = GAIN_POINTS + points
                SELL_PRICE = 0
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "SELL_STOPLOSS")
                
            if BUY and row.Close < BUY_STOPLOSS:
                points = BUY_STOPLOSS - BUY_PRICE
                GAIN_POINTS = GAIN_POINTS + points
                BUY_PRICE = 0
                SELL = False
                BUY = False
                Total_Signals += 1
                Total_Sell_Signal += 1
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "BUY_STOPLOSS")
    
            if row.Close < SMA6_lower_band and \
                    row.Index.time() < entry_close_time and \
                    SELL is False:
                SELL_PRICE = row.Close
                SELL_STOPLOSS = SELL_PRICE + STOPLOSS
                if BUY_PRICE > 0:
                    points = SELL_PRICE - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                SELL = True
                BUY = False
                Total_Sell_Signal += 1
                Total_Signals += 1
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "SELL ENTRY")
                
            if row.Close > SMA6_upper_band and \
                    row.Index.time() < entry_close_time and \
                    BUY is False:
                BUY_PRICE = row.Close
                BUY_STOPLOSS = BUY_PRICE - STOPLOSS
                if SELL_PRICE > 0:
                    points = SELL_PRICE - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                BUY = True
                SELL = False
                Total_Buy_Signal += 1
                Total_Signals += 1
                print(row.Index, "\t", row.Close,  "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "BUY ENTRY")
            
            if row.Index.time() >= close_time:
                if SELL:
                    SELL = False
                    BUY = False
                    Total_Signals += 1
                    Total_Buy_Signal += 1
                    points = SELL_PRICE - row.Close
                    GAIN_POINTS = GAIN_POINTS + points
                    SELL_PRICE = 0
                    print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "BUY_DAYCLOSE")

                if BUY:
                    SELL = False
                    BUY = False
                    Total_Signals += 1
                    Total_Buy_Signal += 1
                    points = row.Close - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                    SELL_PRICE = 0
                    print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "SELL_DAYCLOSE")
                    
                    
    
    print("\n\nTotal Signals:", Total_Signals)
    print("Total Sell Signals:", Total_Buy_Signal)
    print("Total Buy Signals:", Total_Sell_Signal)
    print("Total Points Gained:", GAIN_POINTS)

In [209]:
backtest_Intraday_5min_strategy(today_df_bn5)

Time			 Close		 SMA6		 SMA21		 SIGNAL	 TYPE
2020-09-07 10:05:00 	 23081.45 	 23022.58 	 22999.15 	 BUY	 BUY ENTRY
2020-09-07 10:45:01 	 22953.5 	 23015.58 	 23002.91 	 SELL	 BUY_STOPLOSS
2020-09-07 10:45:01 	 22953.5 	 23015.58 	 23002.91 	 SELL	 SELL ENTRY
2020-09-07 11:15:00 	 23043.0 	 22984.07 	 23004.6 	 BUY	 BUY ENTRY
2020-09-07 14:25:00 	 22945.05 	 23001.95 	 22997.38 	 SELL	 SELL ENTRY
2020-09-07 15:15:00 	 23030.0 	 22920.6 	 22939.05 	 BUY	 BUY_DAYCLOSE


Total Signals: 6
Total Sell Signals: 3
Total Buy Signals: 3
Total Points Gained: -372.40000000000146
